# Kaggle House Prices Challenge

## House Prices: Advanced Regression Techniques

Predict sales prices and practice feature engineering, RFs, and gradient boosting

(Link: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview)

First, we start by importing the libraries needed for this project.

In [20]:
import pandas as pd
import numpy as np

Loading the data into dataframes.

In [21]:
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")

In [22]:
test.shape, train.shape

((1459, 80), (1460, 81))

In [7]:
test.columns, train.columns

(Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
        'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
        'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
        'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
        'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
        'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
        'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
        'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
        'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
        'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
        'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
        'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
        'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
        'GarageCond

We can see that the test dataset contains one more variable compared to the train dataset - which is the "SalePrice" variable. In our analysis / prediction this serves as the dependent variable we want to predict given the houses' characteristics.

In [7]:
train.SalePrice.describe()

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

## Data preprocessing and cleaning

We'll start the Data Cleaning by checking if the dependent variable in the test dataset contains any missing values.

In [18]:
train.SalePrice.isnull().sum()

0

All observations contain data for the target variable, therefore we can continue by taking a look at all the other variables contained in the train and test dataset.

In [9]:
miss_count_train = train.isnull().sum().sort_values(ascending=False)
perc_miss_train = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missings_train = pd.concat([miss_count_train, perc_miss_train], axis=1, keys=["Total", "Percent"])

miss_count_test = test.isnull().sum().sort_values(ascending=False)
perc_miss_test = (test.isnull().sum() / test.isnull().count()).sort_values(ascending=False)
missings_test = pd.concat([miss_count_test, perc_miss_test], axis=1, keys=["Total", "Percent"])

In [10]:
missings_train.head(20)

,Total,Percent
PoolQC,1453,0.995205
MiscFeature,1406,0.963014
Alley,1369,0.937671
Fence,1179,0.807534
FireplaceQu,690,0.472603
LotFrontage,259,0.177397
GarageCond,81,0.055479
GarageType,81,0.055479
GarageYrBlt,81,0.055479
GarageFinish,81,0.055479


In [78]:
missings_test.head(20)

,Total,Percent
PoolQC,1456,0.997944
MiscFeature,1408,0.965045
Alley,1352,0.926662
Fence,1169,0.801234
FireplaceQu,730,0.500343
LotFrontage,227,0.155586
GarageCond,78,0.053461
GarageQual,78,0.053461
GarageYrBlt,78,0.053461
GarageFinish,78,0.053461


As a rule of thumb we completely ignore columns that contain at least 15% missing values and will not try to impute the missing values with any kind of computation, e.g. using means. Therefore, we will delete the variables "PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu" and "LotFrontage".

In [23]:
train = train.drop(columns=["PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu", "LotFrontage"])

The variables "GarageCond", "GarageType", "GarageQual", "GarageYrBlt" and "GarageFinish" contain exactly the same number of missing values, which seems kind of odd. Therefore, we'll take a closer look at these variables.

In [6]:
for var in ["GarageCond", "GarageType", "GarageQual", "GarageYrBlt", "GarageFinish"]:
    print(pd.crosstab(index=train[var], columns="count"))

col_0       count
GarageCond       
Ex              2
Fa             35
Gd              9
Po              7
TA           1326
col_0       count
GarageType       
2Types          6
Attchd        870
Basment        19
BuiltIn        88
CarPort         9
Detchd        387
col_0       count
GarageQual       
Ex              3
Fa             48
Gd             14
Po              3
TA           1311
col_0        count
GarageYrBlt       
1900.0           1
1906.0           1
1908.0           1
1910.0           3
1914.0           2
1915.0           2
1916.0           5
1918.0           2
1920.0          14
1921.0           3
1922.0           5
1923.0           3
1924.0           3
1925.0          10
1926.0           6
1927.0           1
1928.0           4
1929.0           2
1930.0           8
1931.0           4
1932.0           3
1933.0           1
1934.0           2
1935.0           4
1936.0           5
1937.0           2
1938.0           3
1939.0           9
1940.0          14
1941.0         

We can see that for "GarageCond" and "GarageQual" the most frequently occurring value is "TA", which means that the condition and quality of the garages are average/typical. We will replace the missing values of these two variables therefore with "TA" as well. The variable "GarageYrBlt" refers to the year in which the garage was built. Since we also have the year in which the houses themselves are built we can drop this variable without losing much explaining information. In addition to that we also drop the "GarageFinish" and "GarageType" variable.

In [24]:
train = train.drop(columns=["GarageYrBlt", "GarageFinish", "GarageType"])
train["GarageCond"] = train.GarageCond.fillna(value="TA")
train["GarageQual"] = train.GarageQual.fillna(value="TA")

In the same way as above we take a closer look at the "Bsmt*" variables.

In [22]:
for var in ["BsmtFinType2", "BsmtExposure", "BsmtCond", "BsmtFinType1", "BsmtQual"]:
    print(pd.crosstab(index=train[var], columns="count"))

col_0         count
BsmtFinType2       
ALQ              19
BLQ              33
GLQ              14
LwQ              46
Rec              54
Unf            1256
col_0         count
BsmtExposure       
Av              221
Gd              134
Mn              114
No              953
col_0     count
BsmtCond       
Fa           45
Gd           65
Po            2
TA         1311
col_0         count
BsmtFinType1       
ALQ             220
BLQ             148
GLQ             418
LwQ              74
Rec             133
Unf             430
col_0     count
BsmtQual       
Ex          121
Fa           35
Gd          618
TA          649


We delete the "BsmtFinType*" variables since these are highly subjective and do not add much information to our model. The missing values of "BsmtCond" will be imputed with the most common value "TA". The rows containing missing values for "BsmtQual" and "BsmtExposure" will be deleted from the dataset.

In [25]:
train = train.drop(columns=["BsmtFinType1", "BsmtFinType2"])
train.BsmtCond = train["BsmtCond"].fillna(value="TA")
for var in ["BsmtQual", "BsmtExposure"]:
    train = train.drop(train.loc[train[var].isnull()].index)

The variable "Electrical" contains only 1 missing value, therefore we only delete this specific row of data. We proceed in the same way with "MasVnrType" and "MasVnrArea".

In [27]:
for var in ["Electrical", "MasVnrType", "MasVnrArea"]:
    train = train.drop(train.loc[train[var].isnull()].index)

Running the above code again to check if all missing values are deleted.

In [28]:
miss_count_train = train.isnull().sum().sort_values(ascending=False)
perc_miss_train = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missings_train = pd.concat([miss_count_train, perc_miss_train], axis=1, keys=["Total", "Percent"])
missings_train

,Total,Percent
SalePrice,0,0.0
ExterQual,0,0.0
RoofStyle,0,0.0
RoofMatl,0,0.0
Exterior1st,0,0.0
Exterior2nd,0,0.0
MasVnrType,0,0.0
MasVnrArea,0,0.0
ExterCond,0,0.0
YearBuilt,0,0.0


This looks good.

We now handled all the missing data on the training set. As the next step we will clean the test data. Since we want to evaluate our model on Kaggle after finishing the modeling, we cannot drop any observations because we need predicted house prices for each row of the test data. Therefore, we will impute the missing data in the test dataset with the most frequent category for categorical features and the mean for numerical features.

In [32]:
test = test.drop(columns=["PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu", "LotFrontage"])
test = test.drop(columns=["GarageYrBlt", "GarageFinish", "GarageType"])
test["GarageCond"] = test.GarageCond.fillna(value="TA")
test["GarageQual"] = test.GarageQual.fillna(value="TA")
test = test.drop(columns=["BsmtFinType1", "BsmtFinType2"])
test.BsmtCond = test["BsmtCond"].fillna(value="TA")


In [33]:
test.shape, train.shape

((1459, 69), (1413, 70))

In [45]:
# categorical:
for var in ["BsmtExposure", "BsmtQual", "MasVnrType", "MSZoning", "Utilities", "Functional", "SaleType", "Exterior2nd", "Exterior1st", "KitchenQual"]:
    test[var] = test[var].fillna(value=test[var].value_counts().index[0])

# numerical
for var in ["MasVnrArea", "BsmtHalfBath", "BsmtFullBath", "BsmtUnfSF", "GarageArea", "GarageCars", "BsmtFinSF1", "BsmtFinSF2", "TotalBsmtSF"]:
    test[var] = test[var].fillna(test[var].mean())

In [46]:
miss_count_test = test.isnull().sum().sort_values(ascending=False)
perc_miss_test = (test.isnull().sum() / test.isnull().count()).sort_values(ascending=False)
missings_test = pd.concat([miss_count_test, perc_miss_test], axis=1, keys=["Total", "Percent"])
missings_test

,Total,Percent
SaleCondition,0,0.0
MasVnrArea,0,0.0
YearRemodAdd,0,0.0
RoofStyle,0,0.0
RoofMatl,0,0.0
Exterior1st,0,0.0
Exterior2nd,0,0.0
MasVnrType,0,0.0
ExterQual,0,0.0
BsmtUnfSF,0,0.0


In [47]:
test.shape

(1459, 69)

Perfect. We do not have any missing values in our test set anymore and have kept all observations, for which we gonna predict the House Sale Price.

## Model selection 

We continue by building a Machine Learning Pipeline using Scikit Learn. A pipeline object sequentially applies a list of transformers and a final estimator. 


We will play around with different algorithms, tune their hyperparameters using Cross Validation and pick the best performing one.

To make predictions and to fit models, the last step that has to be done is to convert all categorical features into numeric ones. This way scikit-learn can handle them. We do this by using pandas get_dummies() function.

In [51]:
train = pd.get_dummies(train, drop_first=True)
test = pd.get_dummies(test, drop_first=True)
train.shape, test.shape

((1413, 211), (1459, 198))

Before we can work with our data, we first need to create separate dataframes containing our feature variables and the target variable. This needs to be done only for our training data since it is our aim to predict SalePrice for the test data, which is why it is not contained in this data.

In [52]:
y = train.SalePrice.values
X = train.drop("SalePrice", axis=1).values

Before training and tuning our models we separate the data into training and test data to evaluate our models.

In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [54]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1130, 210), (283, 210), (1130,), (283,))

 ### k-nearest Neighbors (KNN)

We will start with a relatively simple algorithm - k-nearest Neighbor or KNN.

In [55]:
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

Looking at the data, we notice that are our feature variables' ranges vary substantially between each other. Therefore we will add a transformer to our pipeline which standardizes the data. Standardization centers each variable around zero with unit variance. This is done by subtracting the means from each feature and dividing by its standard deviation.

After that we instantiate our KNN estimator, create a list containing the steps applied by the pipeline and then defining the pipeline.

In [56]:
# instantiate the scaling transformator 
scaler = preprocessing.StandardScaler()
# instantiate the KNN estimator
knn = KNeighborsRegressor(n_neighbors=10)
# creating a list containing the steps the pipeline is to apply
steps_knn =  [("scaler", scaler), ("knn", knn)]
# define the pipeline object
pipeline_knn = Pipeline(steps_knn)

The KNN algorithm has one parameter that can and should be tuned, which is the number of neighbors that should be considered. We will therefore define a dictionary containing all hyperparameters that should be tuned and define the different values that should be tested.

In [57]:
neighbors = {"knn__n_neighbors":list(range(1,21))}

Next we are gonna set up our Cross Validation (CV) object using 5-fold CV and fit it to our data.

In [58]:
cv_knn = GridSearchCV(pipeline_knn, neighbors, cv=5)

In [59]:
cv_knn.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('knn', KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=10, p=2,
          weights='uniform'))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'knn__n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [60]:
cv_knn.best_params_

{'knn__n_neighbors': 12}

In [61]:
print(cv_knn.refit)

True


From the above output we conclude that the default of the argument "refit" is True. This means that, by default, our CV pipeline automatically refits the model on the entire training set using the best parameters found by CV, which is 12 in our case. Therefore we can now directly use the CV object to make predictions for unseen data.

As in the Kaggle challenge we will use the Root mean squared error metric to evaluate our model's performance on the test set. To do this we first need to predict the sales price for the unseen test data.

In [62]:
from sklearn.metrics import mean_squared_error
y_pred = cv_knn.predict(X_test)
rmse_knn = np.sqrt(mean_squared_error(np.log(y_test), np.log(y_pred)))
print("The KNN algorithm with " + str(cv_knn.best_params_) + " yields a RMSE of: " + str(rmse_knn))

The KNN algorithm with {'knn__n_neighbors': 12} yields a RMSE of: 0.197561198068067


### Random forest

After trying out the KNN algorithm, we now continue with the Random Forest Algorithm. Using similar steps as before we will build up a pipeline object applying the transformations and the estimation on our dataset sequentially automatically.

In [65]:
from sklearn.ensemble import RandomForestRegressor

In [96]:
# instantiate the RandomForest Regressor
rf_reg = RandomForestRegressor(random_state=123)
# creating a list containing the steps the pipeline should apply
steps_rf = [("scaler", scaler), ("rf_reg", rf_reg)]
# create the pipeline object
pipeline_rf = Pipeline(steps_rf)

For RandomForests there are a large number of Hyperparameters that can be tuned. In this project we are going to tune the number of trees in the random forest [n_estimators], the number of features considered at every split [max_features], the maximum number of levels in a tree [max_depth], the mininum number of samples required to split a node [min_samples_split], the minimum number of observations required at each leaf node [min_samples_leaf] and if bootstrap should be used training each tree [bootstrap]. 

We first define the ranges for each hyperparameter that should be considered in our CV tuning and then create our search grid containing the before created lists.

In [97]:
n_estimators = list(np.arange(200, 2001, 200))
max_features = ["auto", "sqrt", "log2"]
max_depth = list(np.arange(10, 101, 10))
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4, 8]
bootstrap = [True, False]
param_dist = {"rf_reg__n_estimators": n_estimators,
              "rf_reg__max_features": max_features,
              "rf_reg__max_depth": max_depth,
              "rf_reg__min_samples_split": min_samples_split,
              "rf_reg__min_samples_leaf": min_samples_leaf,
              "rf_reg__bootstrap": bootstrap}

Testing all possible combinations of hyperparameters would ammount to testing $10*3*11*3*3*2 = 5,940$ combinations, instead of testing all of these we will use RandomizedSearchCV to select randomly from our defined distributions which combinations are tested. 

In [98]:
from sklearn.model_selection import RandomizedSearchCV
cv_random_rf = RandomizedSearchCV(pipeline_rf, param_dist, cv=3, n_iter=100)

Just as before with the KNN algorithm we now can fit the pipeline to our data.

In [99]:
cv_random_rf.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
cv_random_rf.best_params_

Based on the chosen parameters from RandomizedSearchCV we can now manually decrease the range of the hyperparameters to be tested and use GridSearchCV as before to find the best parameters for our model.